In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays
import scipy as sp
import pandas as pd
import pickle
import h5py
import time

In [2]:
bw.projects.set_current('iw_integration')

# Save and load functions with pickle

In [3]:
def pickle_save(file_path_root,object_name_to_save,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    pickle.dump( object_name_to_save, open( complete_file_path, "wb" ) )
    
    return;




def pickle_load(file_path_root,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    return pickle.load( open( complete_file_path, "rb" ) );
    


In [ ]:
#file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
#object_name_to_save=object_to_save
#file_name='object_to_save'

pickle_save(file_path_root,object_name_to_save,file_name)

object_to_load=pickle_load(file_path_root,file_name)

# Save and load LCA object for an activity in eiv33

Pick a random process in eiv33 and construct an LCA object

In [22]:
DB_eiv33=bw.Database('ecoinvent 3.3 cutoff')

In [65]:
act_random_eiv33=DB_eiv33.random()

In [4]:
def create_LCA_object(act,fu,ic_name=None):
    LCA_object=bw.LCA({act:fu},ic_name)
    return LCA_object;

In [67]:
act=act_random_eiv33
fu=1

LCA_object=create_LCA_object(act,fu,ic_name=None)

Create A and B matrices with .load_lci_data() and save the resulting objects with pickle

In [5]:
def create_A_B(LCA_obj,params_only=False, matrix_only=False):
    LCA_obj.load_lci_data(fix_dictionaries=False)
    dict_load_lci_data_objects={}
    
    if params_only==True:
        dict_load_lci_data_objects={'bio_params':LCA_obj.bio_params,
                               'tech_params':LCA_obj.tech_params}
    elif matrix_only==True:
        dict_load_lci_data_objects={'biosphere_matrix':LCA_obj.biosphere_matrix,
                               'technosphere_matrix':LCA_obj.technosphere_matrix}
    
    else:
        dict_load_lci_data_objects={'bio_params':LCA_obj.bio_params,
                               'tech_params':LCA_obj.tech_params,
                               'biosphere_dict':LCA_obj.biosphere_dict,
                               'activity_dict':LCA_obj.activity_dict,
                               'product_dict':LCA_obj.product_dict,
                               'biosphere_matrix':LCA_obj.biosphere_matrix,
                               'technosphere_matrix':LCA_obj.technosphere_matrix}
    
    return dict_load_lci_data_objects;

In [69]:
dict_A_B=create_A_B(LCA_object)

In [70]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B
file_name='dict_A_B_random_eiv33'

pickle_save(file_path_root,object_name_to_save,file_name)

Import the LCA object and finish the calculation

In [71]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B
file_name='dict_A_B_random_eiv33'

dict_A_B_load=pickle_load(file_path_root,file_name)

In [6]:
def load_A_B_to_LCA_object(dict_A_B,act,fu,ic_name=None):
    
    LCA_obj=create_LCA_object(act,fu,ic_name)
    
    #This function recreates the objects created during the .load_lci_data()
    
    #to replace builder.build(self.database_filepath) in .load_lci_data()
    LCA_obj.bio_params=dict_A_B['bio_params']
    LCA_obj.tech_params=dict_A_B['tech_params']
    LCA_obj.biosphere_dict=dict_A_B['biosphere_dict']
    LCA_obj.activity_dict=dict_A_B['activity_dict']
    LCA_obj.product_dict=dict_A_B['product_dict']
    LCA_obj.biosphere_matrix=dict_A_B['biosphere_matrix']
    LCA_obj.technosphere_matrix=dict_A_B['technosphere_matrix']

    LCA_obj.fix_dictionaries() #function in .load_lci_data()
    
    return LCA_obj;

In [73]:
ic_name=('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017','Ecosystem Quality','Land transformation, biodiversity, GLO, with uncert')

LCA_object_load=load_A_B_to_LCA_object(dict_A_B_load,act,fu,ic_name)

In [7]:
def finish_LCA_object(LCA_obj):

    #The beginning of .lci() function replace by the load_A_B_to_LCA_object function
    
    LCA_obj.build_demand_array() #function in .lci() after calling .load_lci_data()
    LCA_obj.lci_calculation() #function in .lci() after calling .load_lci_data()
    LCA_obj.lcia()
    
    return;

In [75]:
finish_LCA_object(LCA_object_load)

In [76]:
LCA_object_load.score

-0.00032472068199512565

## Comparing all created objects in LCA object to test if everything needs to be stored

In [51]:
act_random_eiv33_1=DB_eiv33.random()
act_random_eiv33_2=DB_eiv33.random()
act_1=act_random_eiv33_2
act_2=act_random_eiv33_2
fu=1

LCA_object_1=create_LCA_object(act_1,fu,ic_name=None)
LCA_object_2=create_LCA_object(act_2,fu,ic_name=None)
dict_A_B_1=create_A_B(LCA_object_1)
dict_A_B_2=create_A_B(LCA_object_2)


In [53]:
(dict_A_B_1['bio_params']!=dict_A_B_2['bio_params'],
 dict_A_B_1['tech_params']!=dict_A_B_2['tech_params'],
 dict_A_B_1['biosphere_dict']!=dict_A_B_2['biosphere_dict'],
 dict_A_B_1['activity_dict']!=dict_A_B_2['activity_dict'],
 dict_A_B_1['product_dict']!=dict_A_B_2['product_dict'],
 dict_A_B_1['biosphere_matrix']!=dict_A_B_2['biosphere_matrix'],
 dict_A_B_1['technosphere_matrix']!=dict_A_B_2['technosphere_matrix'])

(array([ True,  True,  True, ...,  True,  True,  True], dtype=bool),
 array([ True,  True,  True, ...,  True,  True,  True], dtype=bool),
 False,
 False,
 False,
 <1960x13831 sparse matrix of type '<class 'numpy.bool_'>'
 	with 0 stored elements in Compressed Sparse Row format>,
 <13831x13831 sparse matrix of type '<class 'numpy.bool_'>'
 	with 0 stored elements in Compressed Sparse Row format>)

In [61]:
biosphere_mat_comparison=LCA_object_1.biosphere_matrix.data!=LCA_object_2.biosphere_matrix.data
technosphere_mat_comparison=LCA_object_1.technosphere_matrix.data!=LCA_object_2.technosphere_matrix.data

In [62]:
for comp in biosphere_mat_comparison:
    if comp==True:
        print('biosphere matrices are different')
        
for comp in technosphere_mat_comparison:
    if comp==True:
        print('biosphere matrices are different')

Only bio_params and tech_params need to be stored, the other are the same for all activities in eiv33! 
--> around 20MB instead of 30MB for all the objects per activity per iteration

In [63]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B_1['bio_params']
file_name='dict_A_B_1_bio_params'

pickle_save(file_path_root,object_name_to_save,file_name)

In [64]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_A_B_1['tech_params']
file_name='dict_A_B_1_tech_params'

pickle_save(file_path_root,object_name_to_save,file_name)

Write a code to only generate bio_params and tech_params? Ça l'air compliqué...

# Monte Carlo for LCI for an activity + storage as dictionary

About objects needed to recreate an LCA object:
-*biosphere_dict, activity_dict, product_dict* are the same for the entire DB
-*bio_params, tech_params* remain the same for an activity whatever the MC iteration
-*biosphere_matrix, technosphere_matrix* change for every MC iteration

In [8]:
DB_eiv33=bw.Database('ecoinvent 3.3 cutoff')

In [9]:
act=DB_eiv33.random()
fu=1
iterations=10
matrix_only=True

start = time.time()
LCA_obj=create_LCA_object(act,fu,ic_name=None)
end = time.time()
print("Calcul en {} secondes".format(end - start))


start = time.time()
dict_A_B=create_A_B(LCA_obj)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 0.0 secondes
Calcul en 0.29302096366882324 secondes


In [10]:
act.as_dict()

{'activity': '1613f0fc-4585-4505-b275-04164e0e6976',
 'activity type': 'ordinary transforming activity',
 'authors': {'data entry': {'email': 'support@ecoinvent.org',
   'name': '[System]'},
  'data generator': {'email': 'psi@ecoinvent.org', 'name': 'Roberto Dones'}},
 'classifications': [('EcoSpold01Categories', 'nuclear power/waste treatment'),
  ('ISIC rev.4 ecoinvent',
   '4290:Construction of other civil engineering projects')],
 'code': '076b954dc63a578bb17c19d13cab296f',
 'comment': 'The Swiss interim storage for nuclear waste Zwilag has been modelled using two equal modules for the infrastructure. The reason is mainly due to the need within the ecoinvent scheme to trace the waste streams which end in the two final repositories for conditioned radioactive wastes. These two streams are the LLW: Low and intermediate Level long-lived radioactive Waste; and, the SF/HLW/ILW: Spent Fuel (unreprocessed in canisters), High Level radioactive Waste (vitrified in reprocessing), and Interme

In [8]:
def LCI_MonteCarlo_from_LCA_object(act,fu,LCA_obj,iterations,matrix_only=True,seed=None):
    
    LCA_obj.fix_dictionaries()
    
    dict_act_MC_LCA_object={}
    dict_act_MC_LCA_object['iterations']=iterations
    dict_act_MC_LCA_object['activity key']=act.key
    dict_act_MC_LCA_object['classifications']=act.as_dict()['classifications']
    dict_act_MC_LCA_object['functional unit']=fu
    #dict_act_MC_LCA_object['bio_params']=LCA_obj.bio_params
    #dict_act_MC_LCA_object['tech_params']=LCA_obj.tech_params
    dict_act_MC_LCA_object['Monte Carlo LCI results']={} 
    
    
    LCA_obj.tech_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.tech_params, seed=seed)
    LCA_obj.bio_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.bio_params, seed=seed)
    
    for it in range(iterations):
        LCA_obj.rebuild_technosphere_matrix(LCA_obj.tech_rng.next())
        LCA_obj.rebuild_biosphere_matrix(LCA_obj.bio_rng.next())
        
        dict_act_MC_LCA_object['Monte Carlo LCI results'][it]=create_A_B(LCA_obj,matrix_only=matrix_only)
        
    return dict_act_MC_LCA_object;    

In [9]:
from bw2calc.utils import extract_uncertainty_fields as euf

def LCI_MonteCarlo_from_LCA_object_params(act,fu,LCA_obj,iterations,matrix_only=True,seed=None):
    
    LCA_obj.fix_dictionaries()
    
    dict_act_MC_LCA_object={}
    dict_act_MC_LCA_object['iterations']=iterations
    dict_act_MC_LCA_object['activity key']=act.key
    dict_act_MC_LCA_object['classifications']=act.as_dict()['classifications']
    dict_act_MC_LCA_object['functional unit']=fu
    #dict_act_MC_LCA_object['bio_params']=LCA_obj.bio_params
    #dict_act_MC_LCA_object['tech_params']=LCA_obj.tech_params
    dict_act_MC_LCA_object['Monte Carlo LCI results']={}
    dict_act_MC_LCA_object['LCI params values']={}
    
    
    # In order to store LCI parameters values at each iteration
    # useful for sensitivity analysis (Inspired from class ParameterVectorLCA)
    positions = {
    "tech": (0, LCA_obj.tech_params.shape[0]),
    "bio": (
        LCA_obj.tech_params.shape[0],
        LCA_obj.tech_params.shape[0] + LCA_obj.bio_params.shape[0]
        )
    }

    # `euf` is extract_uncertainty_fields; needed because we are
    # "gluing" together arrays with different column numbers and labels
    params = (euf(LCA_obj.tech_params), euf(LCA_obj.bio_params))
    
    LCA_obj.positions = positions
    LCA_obj.params = np.hstack(params)
    LCA_obj.rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.params, seed=seed)
    
    
    #LCA_obj.tech_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.tech_params, seed=seed)
    #LCA_obj.bio_rng = stats_arrays.random.MCRandomNumberGenerator(LCA_obj.bio_params, seed=seed)
    
    for it in range(iterations):
        
        LCA_obj.sample = (LCA_obj.rng.next()).copy()
        LCA_obj.tech_sample=LCA_obj.sample[LCA_obj.positions["tech"][0]:LCA_obj.positions["tech"][1]]
        LCA_obj.bio_sample=LCA_obj.sample[LCA_obj.positions["bio"][0]:LCA_obj.positions["bio"][1]]     
        
        LCA_obj.rebuild_technosphere_matrix(LCA_obj.tech_sample)
        LCA_obj.rebuild_biosphere_matrix(LCA_obj.bio_sample)
        
        dict_act_MC_LCA_object['Monte Carlo LCI results'][it]=create_A_B(LCA_obj,matrix_only=matrix_only)
        dict_act_MC_LCA_object['LCI params values'][it]={'tech_sample':LCA_obj.tech_sample,'bio_sample':LCA_obj.bio_sample}
        
    return dict_act_MC_LCA_object;    

In [19]:
start = time.time()
dict_act_MC_LCA_object_A_B_only_10it_1act=LCI_MonteCarlo_from_LCA_object(act,fu,LCA_obj,iterations,matrix_only,seed=None)
end = time.time()
print("Calcul en {} secondes".format(end - start))

start = time.time()
dict_act_MC_LCA_object_A_B_only_10it_1ct_params=LCI_MonteCarlo_from_LCA_object_params(act,fu,LCA_obj,iterations,matrix_only,seed=None)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 3.642524242401123 secondes
Calcul en 3.694936990737915 secondes


In [14]:
current_path_dir=os.getcwd()
path_for_saving=current_path_dir+"\\Saved objects"
path_for_saving

'C:\\Users\\Laure\\Brightway - Notebooks and code\\Monte-Carlo\\Saved objects'

In [15]:
file_path_root=path_for_saving
object_name_to_save=dict_act_MC_LCA_object_A_B_only_10it_1act
file_name='dict_act_MC_LCA_object_A_B_only_10it_1act'

start = time.time()
pickle_save(file_path_root,object_name_to_save,file_name)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 0.2821519374847412 secondes


In [16]:
file_path_root=path_for_saving
object_name_to_save=dict_act_MC_LCA_object_A_B_only_10it_1ct_params
file_name='dict_act_MC_LCA_object_A_B_only_10it_1ct_params'

start = time.time()
pickle_save(file_path_root,object_name_to_save,file_name)
end = time.time()
print("Calcul en {} secondes".format(end - start))

Calcul en 0.4655475616455078 secondes


In [18]:
dict_act_MC_LCA_object

{'Monte Carlo LCI results': {0: {'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 353369 stored elements in Compressed Sparse Row format>,
   'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 147427 stored elements in Compressed Sparse Row format>},
  1: {'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 353369 stored elements in Compressed Sparse Row format>,
   'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 147427 stored elements in Compressed Sparse Row format>},
  2: {'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 353369 stored elements in Compressed Sparse Row format>,
   'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
   	with 147427 stored elements in Compressed Sparse Row format>},
  3: {'biosphere_matrix': <1960x13831 sparse matrix of 

In [19]:
dict_act_MC_LCA_object_params

{'LCI params values': {0: {'bio_sample': array([  6.80218176e-05,   6.75584498e-06,   2.65238031e-07, ...,
            1.81643585e-04,   4.85339759e-03,   1.40998506e-03]),
   'tech_sample': array([ 1.        ,  1.        ,  1.        , ..., -0.99901086,
           0.95062798,  0.99158311])},
  1: {'bio_sample': array([  9.20649337e-05,   7.72893635e-06,   2.92778103e-07, ...,
            1.24551225e-04,   2.03781673e-03,   1.36074204e-03]),
   'tech_sample': array([ 1.        ,  1.        ,  1.        , ..., -0.99901086,
           0.95062798,  0.99158311])},
  2: {'bio_sample': array([  3.89510175e-05,   4.98103755e-06,   2.72218821e-07, ...,
            2.29476828e-04,   6.14485700e-03,   2.97253798e-03]),
   'tech_sample': array([ 1.        ,  1.        ,  1.        , ..., -0.99901086,
           0.95062798,  0.99158311])},
  3: {'bio_sample': array([  2.85315978e-05,   5.56493061e-06,   2.93496712e-07, ...,
            1.78617835e-04,   3.31160848e-03,   3.67061081e-03]),
   'tech

In [49]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=dict_act_MC_LCA_object
file_name='dict_act_MC_LCA_object_matrix_only'

dict_act_MC_LCA_object_load=pickle_load(file_path_root,file_name)

## Store results with HDF5

In [22]:
LCA_obj_to_be_stored_as_HFD5={'per DB':{'biosphere_dict':LCA_obj.biosphere_dict,
                                        'activity_dict':LCA_obj.activity_dict,
                                        'product_dict':LCA_obj.product_dict},
                              'per activity':{'bio_params':LCA_obj.bio_params,
                                             'tech_params':LCA_obj.tech_params},
                              'per iteration':{'bio_sample':LCA_obj.bio_sample,
                                              'tech_sample':LCA_obj.tech_sample},
                              'per iteration and activity':{'technosphere_matrix':LCA_obj.technosphere_matrix,
                                                           'biosphere_matrix':LCA_obj.biosphere_matrix}}

In [23]:
LCA_obj_to_be_stored_as_HFD5

{'per DB': {'activity_dict': {4959: 0,
   4960: 1,
   4961: 2,
   4962: 3,
   4963: 4,
   4964: 5,
   4965: 6,
   4966: 7,
   4967: 8,
   4968: 9,
   4969: 10,
   4970: 11,
   4971: 12,
   4972: 13,
   4973: 14,
   4974: 15,
   4975: 16,
   4976: 17,
   4977: 18,
   4978: 19,
   4979: 20,
   4980: 21,
   4981: 22,
   4982: 23,
   4983: 24,
   4984: 25,
   4985: 26,
   4986: 27,
   4987: 28,
   4988: 29,
   4989: 30,
   4990: 31,
   4991: 32,
   4992: 33,
   4993: 34,
   4994: 35,
   4995: 36,
   4996: 37,
   4997: 38,
   4998: 39,
   4999: 40,
   5000: 41,
   5001: 42,
   5002: 43,
   5003: 44,
   5004: 45,
   5005: 46,
   5006: 47,
   5007: 48,
   5008: 49,
   5009: 50,
   5010: 51,
   5011: 52,
   5012: 53,
   5013: 54,
   5014: 55,
   5015: 56,
   5016: 57,
   5017: 58,
   5018: 59,
   5019: 60,
   5020: 61,
   5021: 62,
   5022: 63,
   5023: 64,
   5024: 65,
   5025: 66,
   5026: 67,
   5027: 68,
   5028: 69,
   5029: 70,
   5030: 71,
   5031: 72,
   5032: 73,
   5033: 74,
   5034:

In [10]:
import copy

def searching_for_values(old_dict,old_key,current_dict,key_path):
    
    if type(current_dict)!=dict:
        value=current_dict
        path=key_path
        
        del old_dict[old_key]
        
        return (path,value);
    
    elif current_dict=={}:
        del old_dict[old_key]
        
        to_return="leaf with no value"
        
        return to_return;
    
    else:
        keys_list=list(current_dict.keys())
        key=keys_list[0]
        key_path=key_path+'/'+str(key)
        old_dict=current_dict
        old_key=key
        current_dict=current_dict[key]
        
        return searching_for_values(old_dict,old_key,current_dict,key_path);
    
    
def from_multidim_dict_to_dict_with_paths(multidim_dict):

    #create a copy of the dict to explore
    multidim_dict_copy=copy.deepcopy(multidim_dict)

    #create a flat dictionary with path as keys and real data to store as value
    dict_with_paths={}
        
    #create the list of keys in the first dictionary
    keys_list=list(multidim_dict_copy.keys())
    key_path=''
    
    while keys_list!=[]:

        current_dict=multidim_dict_copy
        
        #Search for values in the tree
        old_dict=current_dict
        old_key=keys_list[0]
        value_found=searching_for_values(old_dict,old_key,current_dict,key_path)
        
        print(value_found)
        
        #Write value in dictionary with paths
        if value_found!="leaf with no value":
            dict_with_paths[value_found[0]]=value_found[1]
            
        #update the key list
        key_path=''
        keys_list=list(multidim_dict_copy.keys())
        
    else:
        print("Dictionary with paths as keys as needed in HDF5 format is ready")

    return dict_with_paths; 

In [11]:
# function create a group containing all the information of a csr matrix scipy  

import scipy as sp

def csr_matrix_to_hdf5(csr,hdf5_file_path,group_full_path):
    
    # Retrieve or create the hdf5 file and the groups and subgroups
    hdf5_file=h5py.File(hdf5_file_path,'a')
    group=hdf5_file.require_group(group_full_path)
    
    # Create datasets containing values of csr matrix
    group.create_dataset('data',data=csr.data)
    group.create_dataset('indptr',data=csr.indptr)
    group.create_dataset('indices',data=csr.indices)
    group.attrs['shape']=csr.shape
    
    hdf5_file.close()
    
    return;
    
# function to rebuild csr matrix from hdf5 storage

def hdf5_to_csr_matrix(hdf5_file_path,group_full_path):
    
    # Access hdf5 file and storage group of the csr info
    hdf5_file=h5py.File(hdf5_file_path,'r')
    group=hdf5_file[group_full_path]
    
    #Rebuild csr matrix
    csr=sp.sparse.csr_matrix((group['data'][:],group['indices'][:],group['indptr'][:]), group.attrs['shape'])
    
    return csr;

In [25]:
LCA_obj_with_paths=from_multidim_dict_to_dict_with_paths(LCA_obj_to_be_stored_as_HFD5)

('/per DB/biosphere_dict/1', 0)
('/per DB/biosphere_dict/2', 1)
('/per DB/biosphere_dict/6', 2)
('/per DB/biosphere_dict/7', 3)
('/per DB/biosphere_dict/11', 4)
('/per DB/biosphere_dict/12', 5)
('/per DB/biosphere_dict/14', 6)
('/per DB/biosphere_dict/15', 7)
('/per DB/biosphere_dict/16', 8)
('/per DB/biosphere_dict/19', 9)
('/per DB/biosphere_dict/22', 10)
('/per DB/biosphere_dict/23', 11)
('/per DB/biosphere_dict/24', 12)
('/per DB/biosphere_dict/25', 13)
('/per DB/biosphere_dict/33', 14)
('/per DB/biosphere_dict/34', 15)
('/per DB/biosphere_dict/40', 16)
('/per DB/biosphere_dict/41', 17)
('/per DB/biosphere_dict/42', 18)
('/per DB/biosphere_dict/44', 19)
('/per DB/biosphere_dict/45', 20)
('/per DB/biosphere_dict/46', 21)
('/per DB/biosphere_dict/47', 22)
('/per DB/biosphere_dict/48', 23)
('/per DB/biosphere_dict/51', 24)
('/per DB/biosphere_dict/52', 25)
('/per DB/biosphere_dict/53', 26)
('/per DB/biosphere_dict/54', 27)
('/per DB/biosphere_dict/55', 28)
('/per DB/biosphere_dict/57'

('/per DB/biosphere_dict/1958', 971)
('/per DB/biosphere_dict/1965', 972)
('/per DB/biosphere_dict/1966', 973)
('/per DB/biosphere_dict/1971', 974)
('/per DB/biosphere_dict/1974', 975)
('/per DB/biosphere_dict/1977', 976)
('/per DB/biosphere_dict/1980', 977)
('/per DB/biosphere_dict/1981', 978)
('/per DB/biosphere_dict/1983', 979)
('/per DB/biosphere_dict/1985', 980)
('/per DB/biosphere_dict/1986', 981)
('/per DB/biosphere_dict/1988', 982)
('/per DB/biosphere_dict/1989', 983)
('/per DB/biosphere_dict/1990', 984)
('/per DB/biosphere_dict/1992', 985)
('/per DB/biosphere_dict/1993', 986)
('/per DB/biosphere_dict/1995', 987)
('/per DB/biosphere_dict/1997', 988)
('/per DB/biosphere_dict/1998', 989)
('/per DB/biosphere_dict/2005', 990)
('/per DB/biosphere_dict/2007', 991)
('/per DB/biosphere_dict/2008', 992)
('/per DB/biosphere_dict/2009', 993)
('/per DB/biosphere_dict/2010', 994)
('/per DB/biosphere_dict/2011', 995)
('/per DB/biosphere_dict/2013', 996)
('/per DB/biosphere_dict/2019', 997)
(

('/per DB/activity_dict/5259', 300)
('/per DB/activity_dict/5260', 301)
('/per DB/activity_dict/5261', 302)
('/per DB/activity_dict/5262', 303)
('/per DB/activity_dict/5263', 304)
('/per DB/activity_dict/5264', 305)
('/per DB/activity_dict/5265', 306)
('/per DB/activity_dict/5266', 307)
('/per DB/activity_dict/5267', 308)
('/per DB/activity_dict/5268', 309)
('/per DB/activity_dict/5269', 310)
('/per DB/activity_dict/5270', 311)
('/per DB/activity_dict/5271', 312)
('/per DB/activity_dict/5272', 313)
('/per DB/activity_dict/5273', 314)
('/per DB/activity_dict/5274', 315)
('/per DB/activity_dict/5275', 316)
('/per DB/activity_dict/5276', 317)
('/per DB/activity_dict/5277', 318)
('/per DB/activity_dict/5278', 319)
('/per DB/activity_dict/5279', 320)
('/per DB/activity_dict/5280', 321)
('/per DB/activity_dict/5281', 322)
('/per DB/activity_dict/5282', 323)
('/per DB/activity_dict/5283', 324)
('/per DB/activity_dict/5284', 325)
('/per DB/activity_dict/5285', 326)
('/per DB/activity_dict/5286

('/per DB/activity_dict/6750', 1791)
('/per DB/activity_dict/6751', 1792)
('/per DB/activity_dict/6752', 1793)
('/per DB/activity_dict/6753', 1794)
('/per DB/activity_dict/6754', 1795)
('/per DB/activity_dict/6755', 1796)
('/per DB/activity_dict/6756', 1797)
('/per DB/activity_dict/6757', 1798)
('/per DB/activity_dict/6758', 1799)
('/per DB/activity_dict/6759', 1800)
('/per DB/activity_dict/6760', 1801)
('/per DB/activity_dict/6761', 1802)
('/per DB/activity_dict/6762', 1803)
('/per DB/activity_dict/6763', 1804)
('/per DB/activity_dict/6764', 1805)
('/per DB/activity_dict/6765', 1806)
('/per DB/activity_dict/6766', 1807)
('/per DB/activity_dict/6767', 1808)
('/per DB/activity_dict/6768', 1809)
('/per DB/activity_dict/6769', 1810)
('/per DB/activity_dict/6770', 1811)
('/per DB/activity_dict/6771', 1812)
('/per DB/activity_dict/6772', 1813)
('/per DB/activity_dict/6773', 1814)
('/per DB/activity_dict/6774', 1815)
('/per DB/activity_dict/6775', 1816)
('/per DB/activity_dict/6776', 1817)
(

('/per DB/activity_dict/8127', 3168)
('/per DB/activity_dict/8128', 3169)
('/per DB/activity_dict/8129', 3170)
('/per DB/activity_dict/8130', 3171)
('/per DB/activity_dict/8131', 3172)
('/per DB/activity_dict/8132', 3173)
('/per DB/activity_dict/8133', 3174)
('/per DB/activity_dict/8134', 3175)
('/per DB/activity_dict/8135', 3176)
('/per DB/activity_dict/8136', 3177)
('/per DB/activity_dict/8137', 3178)
('/per DB/activity_dict/8138', 3179)
('/per DB/activity_dict/8139', 3180)
('/per DB/activity_dict/8140', 3181)
('/per DB/activity_dict/8141', 3182)
('/per DB/activity_dict/8142', 3183)
('/per DB/activity_dict/8143', 3184)
('/per DB/activity_dict/8144', 3185)
('/per DB/activity_dict/8145', 3186)
('/per DB/activity_dict/8146', 3187)
('/per DB/activity_dict/8147', 3188)
('/per DB/activity_dict/8148', 3189)
('/per DB/activity_dict/8149', 3190)
('/per DB/activity_dict/8150', 3191)
('/per DB/activity_dict/8151', 3192)
('/per DB/activity_dict/8152', 3193)
('/per DB/activity_dict/8153', 3194)
(

...

('/per DB/product_dict/12023', 7064)
('/per DB/product_dict/12024', 7065)
('/per DB/product_dict/12025', 7066)
('/per DB/product_dict/12026', 7067)
('/per DB/product_dict/12027', 7068)
('/per DB/product_dict/12028', 7069)
('/per DB/product_dict/12029', 7070)
('/per DB/product_dict/12030', 7071)
('/per DB/product_dict/12031', 7072)
('/per DB/product_dict/12032', 7073)
('/per DB/product_dict/12033', 7074)
('/per DB/product_dict/12034', 7075)
('/per DB/product_dict/12035', 7076)
('/per DB/product_dict/12036', 7077)
('/per DB/product_dict/12037', 7078)
('/per DB/product_dict/12038', 7079)
('/per DB/product_dict/12039', 7080)
('/per DB/product_dict/12040', 7081)
('/per DB/product_dict/12041', 7082)
('/per DB/product_dict/12042', 7083)
('/per DB/product_dict/12043', 7084)
('/per DB/product_dict/12044', 7085)
('/per DB/product_dict/12045', 7086)
('/per DB/product_dict/12046', 7087)
('/per DB/product_dict/12047', 7088)
('/per DB/product_dict/12048', 7089)
('/per DB/product_dict/12049', 7090)
(

('/per DB/product_dict/13753', 8794)
('/per DB/product_dict/13754', 8795)
('/per DB/product_dict/13755', 8796)
('/per DB/product_dict/13756', 8797)
('/per DB/product_dict/13757', 8798)
('/per DB/product_dict/13758', 8799)
('/per DB/product_dict/13759', 8800)
('/per DB/product_dict/13760', 8801)
('/per DB/product_dict/13761', 8802)
('/per DB/product_dict/13762', 8803)
('/per DB/product_dict/13763', 8804)
('/per DB/product_dict/13764', 8805)
('/per DB/product_dict/13765', 8806)
('/per DB/product_dict/13766', 8807)
('/per DB/product_dict/13767', 8808)
('/per DB/product_dict/13768', 8809)
('/per DB/product_dict/13769', 8810)
('/per DB/product_dict/13770', 8811)
('/per DB/product_dict/13771', 8812)
('/per DB/product_dict/13772', 8813)
('/per DB/product_dict/13773', 8814)
('/per DB/product_dict/13774', 8815)
('/per DB/product_dict/13775', 8816)
('/per DB/product_dict/13776', 8817)
('/per DB/product_dict/13777', 8818)
('/per DB/product_dict/13778', 8819)
('/per DB/product_dict/13779', 8820)
(

('/per DB/product_dict/15328', 10369)
('/per DB/product_dict/15329', 10370)
('/per DB/product_dict/15330', 10371)
('/per DB/product_dict/15331', 10372)
('/per DB/product_dict/15332', 10373)
('/per DB/product_dict/15333', 10374)
('/per DB/product_dict/15334', 10375)
('/per DB/product_dict/15335', 10376)
('/per DB/product_dict/15336', 10377)
('/per DB/product_dict/15337', 10378)
('/per DB/product_dict/15338', 10379)
('/per DB/product_dict/15339', 10380)
('/per DB/product_dict/15340', 10381)
('/per DB/product_dict/15341', 10382)
('/per DB/product_dict/15342', 10383)
('/per DB/product_dict/15343', 10384)
('/per DB/product_dict/15344', 10385)
('/per DB/product_dict/15345', 10386)
('/per DB/product_dict/15346', 10387)
('/per DB/product_dict/15347', 10388)
('/per DB/product_dict/15348', 10389)
('/per DB/product_dict/15349', 10390)
('/per DB/product_dict/15350', 10391)
('/per DB/product_dict/15351', 10392)
('/per DB/product_dict/15352', 10393)
('/per DB/product_dict/15353', 10394)
('/per DB/pr

('/per DB/product_dict/16732', 11773)
('/per DB/product_dict/16733', 11774)
('/per DB/product_dict/16734', 11775)
('/per DB/product_dict/16735', 11776)
('/per DB/product_dict/16736', 11777)
('/per DB/product_dict/16737', 11778)
('/per DB/product_dict/16738', 11779)
('/per DB/product_dict/16739', 11780)
('/per DB/product_dict/16740', 11781)
('/per DB/product_dict/16741', 11782)
('/per DB/product_dict/16742', 11783)
('/per DB/product_dict/16743', 11784)
('/per DB/product_dict/16744', 11785)
('/per DB/product_dict/16745', 11786)
('/per DB/product_dict/16746', 11787)
('/per DB/product_dict/16747', 11788)
('/per DB/product_dict/16748', 11789)
('/per DB/product_dict/16749', 11790)
('/per DB/product_dict/16750', 11791)
('/per DB/product_dict/16751', 11792)
('/per DB/product_dict/16752', 11793)
('/per DB/product_dict/16753', 11794)
('/per DB/product_dict/16754', 11795)
('/per DB/product_dict/16755', 11796)
('/per DB/product_dict/16756', 11797)
('/per DB/product_dict/16757', 11798)
('/per DB/pr

('/per DB/product_dict/18467', 13508)
('/per DB/product_dict/18468', 13509)
('/per DB/product_dict/18469', 13510)
('/per DB/product_dict/18470', 13511)
('/per DB/product_dict/18471', 13512)
('/per DB/product_dict/18472', 13513)
('/per DB/product_dict/18473', 13514)
('/per DB/product_dict/18474', 13515)
('/per DB/product_dict/18475', 13516)
('/per DB/product_dict/18476', 13517)
('/per DB/product_dict/18477', 13518)
('/per DB/product_dict/18478', 13519)
('/per DB/product_dict/18479', 13520)
('/per DB/product_dict/18480', 13521)
('/per DB/product_dict/18481', 13522)
('/per DB/product_dict/18482', 13523)
('/per DB/product_dict/18483', 13524)
('/per DB/product_dict/18484', 13525)
('/per DB/product_dict/18485', 13526)
('/per DB/product_dict/18486', 13527)
('/per DB/product_dict/18487', 13528)
('/per DB/product_dict/18488', 13529)
('/per DB/product_dict/18489', 13530)
('/per DB/product_dict/18490', 13531)
('/per DB/product_dict/18491', 13532)
('/per DB/product_dict/18492', 13533)
('/per DB/pr

In [26]:
len(LCA_obj_with_paths)

29628

In [12]:
current_path_dir=os.getcwd()
path_for_saving=current_path_dir+"\\Saved objects"
path_for_saving

'C:\\Users\\Laure\\Brightway - Notebooks and code\\Monte-Carlo\\Saved objects'

In [13]:
hdf5_file_name="test_saved_LCA_obj.hdf5"
hdf5_file_path=path_for_saving+"\\"+hdf5_file_name

In [47]:
import scipy

start = time.time()

hdf5_file=h5py.File(hdf5_file_path,'a')

for path in LCA_obj_with_paths:
    
    print(path)
    
    LCA_obj_to_store=LCA_obj_with_paths[path]
    
    if type(LCA_obj_to_store)==scipy.sparse.csr.csr_matrix:
        csr_matrix_to_hdf5(LCA_obj_to_store,hdf5_file_path,path)
        
    else:
        # Retrieve or create the hdf5 file and the groups and subgroups
        #group=hdf5_file.require_group(path)

        # Create datasets containing LCA object to store
        hdf5_file.create_dataset(path,data=LCA_obj_to_store)

hdf5_file.close()

end = time.time()
print("Calcul en {} secondes".format(end - start))   

/per DB/biosphere_dict/1
/per DB/biosphere_dict/2
/per DB/biosphere_dict/6
/per DB/biosphere_dict/7
/per DB/biosphere_dict/11
/per DB/biosphere_dict/12
/per DB/biosphere_dict/14
/per DB/biosphere_dict/15
/per DB/biosphere_dict/16
/per DB/biosphere_dict/19
/per DB/biosphere_dict/22
/per DB/biosphere_dict/23
/per DB/biosphere_dict/24
/per DB/biosphere_dict/25
/per DB/biosphere_dict/33
/per DB/biosphere_dict/34
/per DB/biosphere_dict/40
/per DB/biosphere_dict/41
/per DB/biosphere_dict/42
/per DB/biosphere_dict/44
/per DB/biosphere_dict/45
/per DB/biosphere_dict/46
/per DB/biosphere_dict/47
/per DB/biosphere_dict/48
/per DB/biosphere_dict/51
/per DB/biosphere_dict/52
/per DB/biosphere_dict/53
/per DB/biosphere_dict/54
/per DB/biosphere_dict/55
/per DB/biosphere_dict/57
/per DB/biosphere_dict/58
/per DB/biosphere_dict/59
/per DB/biosphere_dict/60
/per DB/biosphere_dict/63
/per DB/biosphere_dict/64
/per DB/biosphere_dict/67
/per DB/biosphere_dict/69
/per DB/biosphere_dict/70
/per DB/biospher

/per DB/biosphere_dict/3357
/per DB/biosphere_dict/3359
/per DB/biosphere_dict/3361
/per DB/biosphere_dict/3362
/per DB/biosphere_dict/3363
/per DB/biosphere_dict/3364
/per DB/biosphere_dict/3365
/per DB/biosphere_dict/3368
/per DB/biosphere_dict/3376
/per DB/biosphere_dict/3379
/per DB/biosphere_dict/3383
/per DB/biosphere_dict/3384
/per DB/biosphere_dict/3389
/per DB/biosphere_dict/3390
/per DB/biosphere_dict/3392
/per DB/biosphere_dict/3393
/per DB/biosphere_dict/3394
/per DB/biosphere_dict/3395
/per DB/biosphere_dict/3396
/per DB/biosphere_dict/3398
/per DB/biosphere_dict/3399
/per DB/biosphere_dict/3401
/per DB/biosphere_dict/3402
/per DB/biosphere_dict/3403
/per DB/biosphere_dict/3405
/per DB/biosphere_dict/3408
/per DB/biosphere_dict/3409
/per DB/biosphere_dict/3410
/per DB/biosphere_dict/3412
/per DB/biosphere_dict/3414
/per DB/biosphere_dict/3418
/per DB/biosphere_dict/3419
/per DB/biosphere_dict/3421
/per DB/biosphere_dict/3426
/per DB/biosphere_dict/3428
/per DB/biosphere_di

/per DB/activity_dict/5651
/per DB/activity_dict/5652
/per DB/activity_dict/5653
/per DB/activity_dict/5654
/per DB/activity_dict/5655
/per DB/activity_dict/5656
/per DB/activity_dict/5657
/per DB/activity_dict/5658
/per DB/activity_dict/5659
/per DB/activity_dict/5660
/per DB/activity_dict/5661
/per DB/activity_dict/5662
/per DB/activity_dict/5663
/per DB/activity_dict/5664
/per DB/activity_dict/5665
/per DB/activity_dict/5666
/per DB/activity_dict/5667
/per DB/activity_dict/5668
/per DB/activity_dict/5669
/per DB/activity_dict/5670
/per DB/activity_dict/5671
/per DB/activity_dict/5672
/per DB/activity_dict/5673
/per DB/activity_dict/5674
/per DB/activity_dict/5675
/per DB/activity_dict/5676
/per DB/activity_dict/5677
/per DB/activity_dict/5678
/per DB/activity_dict/5679
/per DB/activity_dict/5680
/per DB/activity_dict/5681
/per DB/activity_dict/5682
/per DB/activity_dict/5683
/per DB/activity_dict/5684
/per DB/activity_dict/5685
/per DB/activity_dict/5686
/per DB/activity_dict/5687
/

/per DB/activity_dict/7127
/per DB/activity_dict/7128
/per DB/activity_dict/7129
/per DB/activity_dict/7130
/per DB/activity_dict/7131
/per DB/activity_dict/7132
/per DB/activity_dict/7133
/per DB/activity_dict/7134
/per DB/activity_dict/7135
/per DB/activity_dict/7136
/per DB/activity_dict/7137
/per DB/activity_dict/7138
/per DB/activity_dict/7139
/per DB/activity_dict/7140
/per DB/activity_dict/7141
/per DB/activity_dict/7142
/per DB/activity_dict/7143
/per DB/activity_dict/7144
/per DB/activity_dict/7145
/per DB/activity_dict/7146
/per DB/activity_dict/7147
/per DB/activity_dict/7148
/per DB/activity_dict/7149
/per DB/activity_dict/7150
/per DB/activity_dict/7151
/per DB/activity_dict/7152
/per DB/activity_dict/7153
/per DB/activity_dict/7154
/per DB/activity_dict/7155
/per DB/activity_dict/7156
/per DB/activity_dict/7157
/per DB/activity_dict/7158
/per DB/activity_dict/7159
/per DB/activity_dict/7160
/per DB/activity_dict/7161
/per DB/activity_dict/7162
/per DB/activity_dict/7163
/

/per DB/activity_dict/8147
/per DB/activity_dict/8148
/per DB/activity_dict/8149
/per DB/activity_dict/8150
/per DB/activity_dict/8151
/per DB/activity_dict/8152
/per DB/activity_dict/8153
/per DB/activity_dict/8154
/per DB/activity_dict/8155
/per DB/activity_dict/8156
/per DB/activity_dict/8157
/per DB/activity_dict/8158
/per DB/activity_dict/8159
/per DB/activity_dict/8160
/per DB/activity_dict/8161
/per DB/activity_dict/8162
/per DB/activity_dict/8163
/per DB/activity_dict/8164
/per DB/activity_dict/8165
/per DB/activity_dict/8166
/per DB/activity_dict/8167
/per DB/activity_dict/8168
/per DB/activity_dict/8169
/per DB/activity_dict/8170
/per DB/activity_dict/8171
/per DB/activity_dict/8172
/per DB/activity_dict/8173
/per DB/activity_dict/8174
/per DB/activity_dict/8175
/per DB/activity_dict/8176
/per DB/activity_dict/8177
/per DB/activity_dict/8178
/per DB/activity_dict/8179
/per DB/activity_dict/8180
/per DB/activity_dict/8181
/per DB/activity_dict/8182
/per DB/activity_dict/8183
/

...

/per DB/product_dict/12799
/per DB/product_dict/12800
/per DB/product_dict/12801
/per DB/product_dict/12802
/per DB/product_dict/12803
/per DB/product_dict/12804
/per DB/product_dict/12805
/per DB/product_dict/12806
/per DB/product_dict/12807
/per DB/product_dict/12808
/per DB/product_dict/12809
/per DB/product_dict/12810
/per DB/product_dict/12811
/per DB/product_dict/12812
/per DB/product_dict/12813
/per DB/product_dict/12814
/per DB/product_dict/12815
/per DB/product_dict/12816
/per DB/product_dict/12817
/per DB/product_dict/12818
/per DB/product_dict/12819
/per DB/product_dict/12820
/per DB/product_dict/12821
/per DB/product_dict/12822
/per DB/product_dict/12823
/per DB/product_dict/12824
/per DB/product_dict/12825
/per DB/product_dict/12826
/per DB/product_dict/12827
/per DB/product_dict/12828
/per DB/product_dict/12829
/per DB/product_dict/12830
/per DB/product_dict/12831
/per DB/product_dict/12832
/per DB/product_dict/12833
/per DB/product_dict/12834
/per DB/product_dict/12835
/

/per DB/product_dict/14021
/per DB/product_dict/14022
/per DB/product_dict/14023
/per DB/product_dict/14024
/per DB/product_dict/14025
/per DB/product_dict/14026
/per DB/product_dict/14027
/per DB/product_dict/14028
/per DB/product_dict/14029
/per DB/product_dict/14030
/per DB/product_dict/14031
/per DB/product_dict/14032
/per DB/product_dict/14033
/per DB/product_dict/14034
/per DB/product_dict/14035
/per DB/product_dict/14036
/per DB/product_dict/14037
/per DB/product_dict/14038
/per DB/product_dict/14039
/per DB/product_dict/14040
/per DB/product_dict/14041
/per DB/product_dict/14042
/per DB/product_dict/14043
/per DB/product_dict/14044
/per DB/product_dict/14045
/per DB/product_dict/14046
/per DB/product_dict/14047
/per DB/product_dict/14048
/per DB/product_dict/14049
/per DB/product_dict/14050
/per DB/product_dict/14051
/per DB/product_dict/14052
/per DB/product_dict/14053
/per DB/product_dict/14054
/per DB/product_dict/14055
/per DB/product_dict/14056
/per DB/product_dict/14057
/

/per DB/product_dict/15301
/per DB/product_dict/15302
/per DB/product_dict/15303
/per DB/product_dict/15304
/per DB/product_dict/15305
/per DB/product_dict/15306
/per DB/product_dict/15307
/per DB/product_dict/15308
/per DB/product_dict/15309
/per DB/product_dict/15310
/per DB/product_dict/15311
/per DB/product_dict/15312
/per DB/product_dict/15313
/per DB/product_dict/15314
/per DB/product_dict/15315
/per DB/product_dict/15316
/per DB/product_dict/15317
/per DB/product_dict/15318
/per DB/product_dict/15319
/per DB/product_dict/15320
/per DB/product_dict/15321
/per DB/product_dict/15322
/per DB/product_dict/15323
/per DB/product_dict/15324
/per DB/product_dict/15325
/per DB/product_dict/15326
/per DB/product_dict/15327
/per DB/product_dict/15328
/per DB/product_dict/15329
/per DB/product_dict/15330
/per DB/product_dict/15331
/per DB/product_dict/15332
/per DB/product_dict/15333
/per DB/product_dict/15334
/per DB/product_dict/15335
/per DB/product_dict/15336
/per DB/product_dict/15337
/

/per DB/product_dict/16697
/per DB/product_dict/16698
/per DB/product_dict/16699
/per DB/product_dict/16700
/per DB/product_dict/16701
/per DB/product_dict/16702
/per DB/product_dict/16703
/per DB/product_dict/16704
/per DB/product_dict/16705
/per DB/product_dict/16706
/per DB/product_dict/16707
/per DB/product_dict/16708
/per DB/product_dict/16709
/per DB/product_dict/16710
/per DB/product_dict/16711
/per DB/product_dict/16712
/per DB/product_dict/16713
/per DB/product_dict/16714
/per DB/product_dict/16715
/per DB/product_dict/16716
/per DB/product_dict/16717
/per DB/product_dict/16718
/per DB/product_dict/16719
/per DB/product_dict/16720
/per DB/product_dict/16721
/per DB/product_dict/16722
/per DB/product_dict/16723
/per DB/product_dict/16724
/per DB/product_dict/16725
/per DB/product_dict/16726
/per DB/product_dict/16727
/per DB/product_dict/16728
/per DB/product_dict/16729
/per DB/product_dict/16730
/per DB/product_dict/16731
/per DB/product_dict/16732
/per DB/product_dict/16733
/

/per DB/product_dict/17796
/per DB/product_dict/17797
/per DB/product_dict/17798
/per DB/product_dict/17799
/per DB/product_dict/17800
/per DB/product_dict/17801
/per DB/product_dict/17802
/per DB/product_dict/17803
/per DB/product_dict/17804
/per DB/product_dict/17805
/per DB/product_dict/17806
/per DB/product_dict/17807
/per DB/product_dict/17808
/per DB/product_dict/17809
/per DB/product_dict/17810
/per DB/product_dict/17811
/per DB/product_dict/17812
/per DB/product_dict/17813
/per DB/product_dict/17814
/per DB/product_dict/17815
/per DB/product_dict/17816
/per DB/product_dict/17817
/per DB/product_dict/17818
/per DB/product_dict/17819
/per DB/product_dict/17820
/per DB/product_dict/17821
/per DB/product_dict/17822
/per DB/product_dict/17823
/per DB/product_dict/17824
/per DB/product_dict/17825
/per DB/product_dict/17826
/per DB/product_dict/17827
/per DB/product_dict/17828
/per DB/product_dict/17829
/per DB/product_dict/17830
/per DB/product_dict/17831
/per DB/product_dict/17832
/

In [14]:
hdf5_file=h5py.File(hdf5_file_path,'a')
#hdf5_file.clear()

In [50]:
[key for key in hdf5_file.keys()]

['per DB', 'per activity', 'per iteration', 'per iteration and activity']

In [53]:
bio_params_saved=hdf5_file['/per activity/bio_params']
bio_params_saved.value

array([ (   1,  5016,    0,    57, 2, 2,   6.14740129e-05,  -9.6968956 ,  0.28495613,  nan,  nan,  nan, False),
       (   1,  5029,    0,    70, 2, 2,   4.99028010e-06, -12.2080183 ,  0.24617067,  nan,  nan,  nan, False),
       (   1,  5100,    0,   141, 2, 2,   2.76879007e-07, -15.09968567,  0.05744563,  nan,  nan,  nan, False),
       ...,
       (4026, 18773, 1958, 13814, 2, 2,   1.80029136e-04,  -8.6223917 ,  0.57445627,  nan,  nan,  nan, False),
       (4029, 14719, 1959,  9760, 2, 2,   3.42019997e-03,  -5.67805624,  0.4669047 ,  nan,  nan,  nan, False),
       (4029, 16287, 1959, 11328, 2, 2,   3.42019997e-03,  -5.67805624,  0.4669047 ,  nan,  nan,  nan, False)],
      dtype=[('input', '<u4'), ('output', '<u4'), ('row', '<u4'), ('col', '<u4'), ('type', 'u1'), ('uncertainty_type', 'u1'), ('amount', '<f4'), ('loc', '<f4'), ('scale', '<f4'), ('shape', '<f4'), ('minimum', '<f4'), ('maximum', '<f4'), ('negative', '?')])

## Retrieve results from HFD5 to dictionnary

In [30]:
def h5py_dataset_iterator(g, prefix=''):
    for key in g.keys():
        item = g[key]
        path = '{}/{}'.format(prefix, key)
        if isinstance(item, h5py.Dataset): # test for dataset
            yield (path, item)
        elif isinstance(item, h5py.Group): # test for group (go down)
            yield from h5py_dataset_iterator(item, path)


def create_multidict_from_path(path, obj, multidict):
    
    number_of_group=path.count('/')
    hierarchic_key_list=[key for key in list(path.rsplit('/', number_of_group)) if key!='']
    last_name=hierarchic_key_list[-1:][0]
    
    multidict_modif=multidict
    
    for key in hierarchic_key_list:
        
        if key!=last_name:
            
            if key not in multidict_modif.keys():
                multidict_modif[key]={}
            multidict_modif=multidict_modif[key]
        
        else:
            multidict_modif[key]=obj
    
    return multidict;
            


def from_hdf5_group_to_multidict(hdf5_file_path,group_full_path):
    
    hdf5_file=h5py.File(hdf5_file_path,'r')
    obj=hdf5_file[group_full_path]
    last_name_group=group_full_path.rsplit('/', 1)[1]
    
    matrix_path_already_done=[]
    
    # Use csr builder if biosphere_matrix or technosphere_matrix and write directly the object
    if '_matrix' in last_name_group:
        obj=hdf5_to_csr_matrix(hdf5_file_path,group_full_path)
        multidict=obj
    
    # if group is a dataset write directly the object
    elif isinstance(obj, h5py.Dataset):
        multidict=obj[()]
        
    else:
        
        multidict={}
        
        for (path, dset) in h5py_dataset_iterator(obj):
            
            if path not in matrix_path_already_done:
            
                if '_matrix' in path:                
                    matrix_path=path.rsplit('_matrix/', 1)[0]+'_matrix'
                    matrix_full_path=group_full_path+matrix_path
                    obj=hdf5_to_csr_matrix(hdf5_file_path,matrix_full_path)

                    multidict=create_multidict_from_path(matrix_path, obj, multidict)
                    matrix_path_already_done.append(matrix_path)

                else:
                    multidict=create_multidict_from_path(path, dset[()], multidict)
                    
    hdf5_file.close()
    
    return multidict;


In [31]:
current_path_dir=os.getcwd()
path_for_saving=current_path_dir+"\\Saved objects"

hdf5_file_name_1="test_saved_LCA_obj.hdf5"
hdf5_file_path=path_for_saving+"\\"+hdf5_file_name_1
group_full_path='/' #if we want to scan the entire file, just put '/'

start = time.time()

multidict_test=from_hdf5_group_to_multidict(hdf5_file_path,group_full_path)

end = time.time()
print("Calcul en {} secondes".format(end - start)) 

multidict_test

Calcul en 5.025772333145142 secondes


{'per DB': {'activity_dict': {'10000': 5041,
   '10001': 5042,
   '10002': 5043,
   '10003': 5044,
   '10004': 5045,
   '10005': 5046,
   '10006': 5047,
   '10007': 5048,
   '10008': 5049,
   '10009': 5050,
   '10010': 5051,
   '10011': 5052,
   '10012': 5053,
   '10013': 5054,
   '10014': 5055,
   '10015': 5056,
   '10016': 5057,
   '10017': 5058,
   '10018': 5059,
   '10019': 5060,
   '10020': 5061,
   '10021': 5062,
   '10022': 5063,
   '10023': 5064,
   '10024': 5065,
   '10025': 5066,
   '10026': 5067,
   '10027': 5068,
   '10028': 5069,
   '10029': 5070,
   '10030': 5071,
   '10031': 5072,
   '10032': 5073,
   '10033': 5074,
   '10034': 5075,
   '10035': 5076,
   '10036': 5077,
   '10037': 5078,
   '10038': 5079,
   '10039': 5080,
   '10040': 5081,
   '10041': 5082,
   '10042': 5083,
   '10043': 5084,
   '10044': 5085,
   '10045': 5086,
   '10046': 5087,
   '10047': 5088,
   '10048': 5089,
   '10049': 5090,
   '10050': 5091,
   '10051': 5092,
   '10052': 5093,
   '10053': 5094,
 

In [23]:
multidict_test.keys()

dict_keys(['per DB', 'per activity', 'per iteration', 'per iteration and activity'])

In [33]:
hdf5_file_name_1="test_saved_LCA_obj.hdf5"
hdf5_file_path=path_for_saving+"\\"+hdf5_file_name_1
group_full_path='/per iteration and activity' #if we want to scan the entire file, just put '/'

start = time.time()

multidict_test_1=from_hdf5_group_to_multidict(hdf5_file_path,group_full_path)

end = time.time()
print("Calcul en {} secondes".format(end - start)) 

multidict_test_1

Calcul en 0.021016836166381836 secondes


{'biosphere_matrix': <1960x13831 sparse matrix of type '<class 'numpy.float64'>'
 	with 353369 stored elements in Compressed Sparse Row format>,
 'technosphere_matrix': <13831x13831 sparse matrix of type '<class 'numpy.float64'>'
 	with 147427 stored elements in Compressed Sparse Row format>}